In [1]:
import numpy as np
def load_glove_embeddings(path):
    word2vec_dict = {}            
    with open(path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                values = line.split()
                word = values[0]
                vector = np.asarray(values[1:], dtype='float32')
                word2vec_dict[word] = vector
            except ValueError:                
                continue    
    return word2vec_dict

path = "glove.840B.300d.txt"
word2vec = load_glove_embeddings(path)
print(len(word2vec))


import pickle

with open('w2v.pkl', 'wb') as f:
    pickle.dump(word2vec, f)

2195884


## --- process the fraction of datasets ---

In [4]:
import pandas as pd
namelist = ['cr', 'trec', 'agnews', 'pc', 'yelp', 'kaggle_med', 'cardio', 'bbc', 'sst2','subj']
for name in namelist:
    df = pd.read_csv(f'data/original/{name}/train.csv')
    df = df.sample(frac=0.5, random_state=100)
    df.to_csv(f'data/original/{name}/train_50.csv', index=False)


    

In [1]:
import torch
torch.cuda.is_available()

True

## --- preprocess the data ---

In [9]:
# load data and remove all rows with nan or empty string or space or NONE or None or none or NaN or nan or NaT or nat or N/A or n/a or NULL or null or Null or nil or NIL or Nil or na or NA or n.a. or N.A. or n.a or N.a or N.A or n.A or n.A. or N.a. or N.A. or n.A. or n.A. or N.A. or n.a. or N.a
# and print rows that removed
import pandas as pd
import numpy as np
import re

def clean(path):
    df = pd.read_csv(path)
    invalid_values = ['', ' ', 'NONE', 'None', 'none', 'NaN', 'nan', 'NaT', 'nat', 'N/A', 'n/a',
                        'NULL', 'null', 'Null', 'nil', 'NIL', 'Nil', 'na', 'NA', 'n.a.', 'N.A.', 'n.a', 'N.a', 'N.A',
                          'n.A', 'n.A.', 'N.a.', 'N.A.', 'n.A.', 'n.A.', 'N.A.', 'n.a.', 'N.a']
    for text in df['text']:
        if text in invalid_values and len(text) < 4:
            # delete the row

            # reset index and save to csv

            
            return df

In [ ]:
namelist = ['cr', 'trec', 'agnews', 'pc', 'yelp', 'kaggle_med', 'cardio', 'bbc', 'sst2','subj']
for name in namelist:
    clean(f'data/original/{name}/train.csv')

In [15]:
import pandas as pd
import numpy as np
import re

def clean(path):
    df = pd.read_csv(path)
    invalid_values = ['', ' ', 'NONE', 'None', 'none', 'NaN', 'nan', 'NaT', 'nat', 'N/A', 'n/a',
                        'NULL', 'null', 'Null', 'nil', 'NIL', 'Nil', 'na', 'NA', 'n.a.', 'N.A.', 'n.a', 'N.a', 'N.A',
                          'n.A', 'n.A.', 'N.a.', 'N.A.', 'n.A.', 'n.A.', 'N.A.', 'n.a.', 'N.a']
    
    invalid_rows = df[df['text'].apply(lambda x: x in invalid_values and len(x) < 4) | df['class'].apply(lambda x: x in invalid_values and len(x) < 4)]

    # Print the removed rows
    print("Removed rows:")
    print(invalid_rows)

In [1]:
import pandas as pd
train_path = 'data/original/pubmed/train.csv'
test_path = 'data/original/pubmed/test.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
train_df['class'] = train_df['class'].map({'OBJECTIVE': 0, 'METHODS': 1, 'RESULTS': 2, 'CONCLUSIONS': 3, 'BACKGROUND': 4})
test_df['class'] = test_df['class'].map({'OBJECTIVE': 0, 'METHODS': 1, 'RESULTS': 2, 'CONCLUSIONS': 3, 'BACKGROUND': 4})
train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)


In [2]:
df

,class,text
0,OBJECTIVE,To investigate the efficacy of 6 weeks of dail...
1,METHODS,A total of 125 patients with primary knee OA w...
2,METHODS,Outcome measures included pain reduction and i...
3,METHODS,Pain was assessed using the visual analog pain...
4,METHODS,Secondary outcome measures included the Wester...
...,...,...
180035,RESULTS,For the absolute change in percent atheroma vo...
180036,RESULTS,"For PAV , a significantly greater percentage o..."
180037,RESULTS,Both strategies had acceptable side effect pro...
180038,CONCLUSIONS,"Compared with standard statin monotherapy , th..."
